# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [30]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [31]:
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio"
req = requests.get(url)

print("O status 200 é equivalente ao status OK. Esta requisição retorna:", req.status_code)

O status 200 é equivalente ao status OK. Esta requisição retorna: 200


In [32]:
dados_json = req.json()

df = pd.json_normalize(dados_json) # ação necessária porque há arrays com tamanhos diferentes nesta Base de Dados

items_df = pd.DataFrame(df["items"].iloc[0]) # novo DataFrame porque todas as informações estavam agrupadas em na primeira coluna como lista de dicionários, impossibilitando trabalhá-la
items_df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10226,Município,Conceição de Macabu,3301405,RJ,None,Arquivado,17944.001149/2008-14,2008-08-14T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,400700.0,0,0,30/10/2008
1,10281,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.001161/2008-29,2008-06-13T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2008 B,Instituição Financeira Nacional,Caixa Econômica Federal,Real,56947800.0,1,0,30/06/2008
2,11501,Município,Belford Roxo,3300456,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001463/2007-16,2007-11-29T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,13200000.0,0,0,28/08/2008


In [33]:
items_df["status"].value_counts() # exibindo em ordem de frequência, logo seriam "Deferido", "Arquivado" e "Regularizado"

Deferido                                                 95
Arquivado                                                53
Regularizado                                             26
Arquivado por decurso de prazo                           24
Deferido (PVL-IF)                                        19
Encaminhado à PGFN com manifestação técnica favorável    14
Arquivado a pedido                                        6
Em retificação pelo interessado                           2
Em retificação pelo interessado (PVL-IF)                  1
Em retificação pelo credor                                1
Name: status, dtype: int64

In [34]:
ano_status = items_df["data_status"].str.split("/", expand=True).get(2) # tive que usar o parâmetro "expand" para percorrer todas as linhas
ano_status

0      2008
1      2008
2      2008
3      2008
4      2011
       ... 
236    2008
237    2019
238    2020
239    2019
240    2020
Name: 2, Length: 241, dtype: object

In [35]:
ano_status.value_counts() # O ano que houve mais solicitações foi 2008

2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2022    12
2011    10
2023     9
2009     9
2015     8
2017     5
2021     5
2002     4
2006     4
2016     4
2018     4
2003     1
2004     1
Name: 2, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [36]:
def obter_dados(UF, tipo_interessado):
    url = f"https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={UF}&tipo_interessado={tipo_interessado}"
    resposta = requests.get(url)
    dados = resposta.json()
    df = pd.json_normalize(dados)
    ndf = pd.DataFrame(df["items"].iloc[0])
    return ndf

In [37]:
MG_est = obter_dados("MG", "Estado")
MG_est["status"].value_counts() # apenas 1 registro

Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado pela STN                                        3
Arquivado                                                 3
Indeferido                                                1
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [38]:
BA_mun = obter_dados("BA", "Município")
BA_mun["interessado"].value_counts() # Camaçari, Litoral Norte, próximo à capital Salvador

Camaçari                  19
Luís Eduardo Magalhães    17
Salvador                  17
Amargosa                  14
Irecê                     14
                          ..
Matina                     1
Boninal                    1
Angical                    1
Ituaçu                     1
Inhambupe                  1
Name: interessado, Length: 289, dtype: int64

In [39]:
BA_est = obter_dados("BA", "Estado")
BA_est.to_csv("dados_solicitacoes_EstadoBA_sem_index_separados.csv", index= False, sep=';')